In [1]:
import os
import glob
import json
import re
import time
from tqdm import tqdm
tqdm.pandas()

import pandas as pd
from pandas.plotting import register_matplotlib_converters
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display
from datetime import datetime, timedelta
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from konlpy.tag import Okt

from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
plt.rcParams['axes.unicode_minus'] = False
# font_path = '/usr/share/fonts/NanumGothic.ttf' 

import warnings
warnings.filterwarnings('ignore')

In [2]:
path = '/data/mountain/'

# Testset

In [3]:
test = pd.read_json(path+'test.json', typ = 'frame')
test

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],70107,,"[398985, 449403, 411543, 528044, 143048, 98020...",6,2012-09-29 01:57:26.000
1,"[나만의Best3, 인디아티스트들의추천음악]",7461,,"[196298, 269984, 267805, 175867, 529244, 63825...",0,2019-12-17 14:06:45.000
2,[드라이브],90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",21,2015-05-23 10:44:48.000
3,[분위기],58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",0,2019-03-14 09:47:34.000
4,[],102395,,"[630683, 481582, 528550, 285114, 506667, 17922...",38,2018-07-11 16:43:32.000
...,...,...,...,...,...,...
10735,[추억],137930,,"[323755, 397594, 445908, 570242, 221853, 20018...",16,2016-04-18 11:02:09.000
10736,"[띵곡의, 우울, 분위기, 드라이브, 산책]",936,,"[105140, 582252, 199262, 422915, 547967, 48791...",1,2020-04-08 07:15:59.000
10737,[기분전환],110589,,"[21976, 207746, 40025, 31635, 567462, 641799, ...",6,2016-06-29 00:57:21.000
10738,[여름],2605,,"[234554, 265033, 507260, 83092, 366757, 497097...",4,2015-06-06 09:52:01.000


### 태그, 곡, 제목이 비어있는 경우가 있는지 확인

In [4]:
# 태그 비어있는 경우 => 25% 이상 존재 
test.apply(lambda x: len(x['tags']), axis=1).describe()

count    10740.000000
mean         1.185940
std          1.567718
min          0.000000
25%          0.000000
50%          1.000000
75%          2.000000
max          5.000000
dtype: float64

In [5]:
# 곡 비어있는 경우 => 25% 미만 존재 
test.apply(lambda x: len(x['songs']), axis=1).describe()

count    10740.000000
mean        18.402980
std         21.649619
min          0.000000
25%          6.000000
50%         11.000000
75%         23.000000
max        100.000000
dtype: float64

In [6]:
# 플레이리스트 제목 비어있는 경우 => 75% 이상 존재 
test.apply(lambda x: len(x['plylst_title']), axis=1).describe()

count    10740.000000
mean         3.380726
std          7.802921
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         40.000000
dtype: float64

In [9]:
# 값이 있으면 1 없으면 0 반환하는 열 추가
temp = test.copy()
temp['ck_tags'] = temp.apply(lambda x: 0 if len(x['tags'])==0 else 1, axis=1)
temp['ck_songs'] = temp.apply(lambda x: 0 if len(x['songs'])==0 else 1, axis=1)
temp['ck_titles'] = temp.apply(lambda x: 0 if len(x['plylst_title'])==0 else 1, axis=1)
temp

,tags,id,plylst_title,songs,like_cnt,updt_date,ck_tags,ck_songs,ck_titles
0,[],70107,,"[398985, 449403, 411543, 528044, 143048, 98020...",6,2012-09-29 01:57:26.000,0,1,0
1,"[나만의Best3, 인디아티스트들의추천음악]",7461,,"[196298, 269984, 267805, 175867, 529244, 63825...",0,2019-12-17 14:06:45.000,1,1,0
2,[드라이브],90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",21,2015-05-23 10:44:48.000,1,1,0
3,[분위기],58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",0,2019-03-14 09:47:34.000,1,1,0
4,[],102395,,"[630683, 481582, 528550, 285114, 506667, 17922...",38,2018-07-11 16:43:32.000,0,1,0
...,...,...,...,...,...,...,...,...,...
10735,[추억],137930,,"[323755, 397594, 445908, 570242, 221853, 20018...",16,2016-04-18 11:02:09.000,1,1,0
10736,"[띵곡의, 우울, 분위기, 드라이브, 산책]",936,,"[105140, 582252, 199262, 422915, 547967, 48791...",1,2020-04-08 07:15:59.000,1,1,0
10737,[기분전환],110589,,"[21976, 207746, 40025, 31635, 567462, 641799, ...",6,2016-06-29 00:57:21.000,1,1,0
10738,[여름],2605,,"[234554, 265033, 507260, 83092, 366757, 497097...",4,2015-06-06 09:52:01.000,1,1,0


In [18]:
# 인풋값과 해당 비율 반환하는 함수
def get_n_pct(x:int, n:int):
    return x, round(x/n*100, 2)

In [25]:
# 태그, 곡, 제목 비어있는 개수 확인
print('모두 비어있는 플리 개수, %:', get_n_pct(temp.query('ck_tags==0 & ck_songs==0 & ck_titles==0').id.count(), len(test)))
print('태그만 비어있는 플리 개수, %:', get_n_pct(temp.query('ck_tags==0 & ck_songs==1 & ck_titles==1').id.count(), len(test)))
print('곡만 비어있는 플리 개수, %:', get_n_pct(temp.query('ck_tags==1 & ck_songs==0 & ck_titles==1').id.count(), len(test)))
print('제목만 비어있는 플리 개수, %:', get_n_pct(temp.query('ck_tags==1 & ck_songs==1 & ck_titles==0').id.count(), len(test)))
print('태그와 곡이 비어있는 플리 개수, %:', get_n_pct(temp.query('ck_tags==0 & ck_songs==0 & ck_titles==1').id.count(), len(test)))

모두 비어있는 플리 개수, %: (2, 0.02)
태그만 비어있는 플리 개수, %: (0, 0.0)
곡만 비어있는 플리 개수, %: (1232, 11.47)
제목만 비어있는 플리 개수, %: (4190, 39.01)
태그와 곡이 비어있는 플리 개수, %: (809, 7.53)


In [26]:
# 태그&곡 비어있는 경우
temp.query('ck_tags==0 & ck_songs==0')

,tags,id,plylst_title,songs,like_cnt,updt_date,ck_tags,ck_songs,ck_titles
17,[],43130,한여름의 센티멘탈 피아노.,[],4,2014-06-04 01:22:05.000,0,0,1
38,[],17669,라붐유명한노래들-좋은노래,[],5,2017-10-10 14:22:18.000,0,0,1
40,[],20179,+안개낀 날씨와 어울리는 신비로운 Pop+,[],3,2015-10-19 17:27:41.000,0,0,1
49,[],133553,들으면 들을수록 더 좋아지는 발라드,[],12,2014-01-28 19:35:34.000,0,0,1
70,[],131124,사랑하는 나의 아버지 나의 맘 받으소서,[],95,2020-04-03 04:19:59.000,0,0,1
...,...,...,...,...,...,...,...,...,...
10667,[],114894,신나게씽씽,[],12,2019-06-25 08:01:39.000,0,0,1
10678,[],90786,크리스마스에 듣기 좋은 발라드,[],28,2018-11-18 19:20:07.000,0,0,1
10686,[],6756,정말추천하는 감성힙합,[],21,2018-11-14 18:48:05.000,0,0,1
10701,[],80375,남자그룹 노래들,[],21,2017-03-07 16:33:22.000,0,0,1
